In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import dvc.api
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder

In [2]:
path = 'data/AdSmartABdata1.csv'
repo = '/home/michael/abtest-mlops'
version = 'v2'

data_url = dvc.api.get_url(
    path = path,
    repo = repo,
    rev=version
    )

In [3]:
df = pd.read_csv(data_url)

FileNotFoundError: [Errno 2] No such file or directory: '../../../../tmp/dvc-storage/1a/e19300fae64a8d41f40122d8f68a53'

In [ ]:
df.head()

In [2]:
df['date']=pd.to_datetime(df['date'])
df.head()

NameError: name 'df' is not defined

In [35]:
# Create a Day of Week feature
df['day_of_week'] = df['date'].dt.day_name()
# Remove Date column
del df['date']
df.head()

,auction_id,experiment,hour,device_make,platform_os,browser,yes,no,day_of_week
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,8,Generic Smartphone,6,Chrome Mobile,0,0,Friday
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,10,Generic Smartphone,6,Chrome Mobile,0,0,Tuesday
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2,E5823,6,Chrome Mobile WebView,0,1,Sunday
3,00187412-2932-4542-a8ef-3633901c98d9,control,15,Samsung SM-A705FN,6,Facebook,0,0,Friday
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,15,Generic Smartphone,6,Chrome Mobile,0,0,Friday


In [36]:
ml_data = df.copy()

In [37]:
ml_data.head()

,auction_id,experiment,hour,device_make,platform_os,browser,yes,no,day_of_week
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,8,Generic Smartphone,6,Chrome Mobile,0,0,Friday
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,10,Generic Smartphone,6,Chrome Mobile,0,0,Tuesday
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2,E5823,6,Chrome Mobile WebView,0,1,Sunday
3,00187412-2932-4542-a8ef-3633901c98d9,control,15,Samsung SM-A705FN,6,Facebook,0,0,Friday
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,15,Generic Smartphone,6,Chrome Mobile,0,0,Friday


In [38]:
ord_enc = OrdinalEncoder()

In [39]:
ml_data['experiment']= ord_enc.fit_transform(ml_data[['experiment']])
ml_data['device_make']= ord_enc.fit_transform(ml_data[['device_make']])
ml_data['browser']= ord_enc.fit_transform(ml_data[['browser']])
ml_data['day_of_week']= ord_enc.fit_transform(ml_data[['day_of_week']])
ml_data.head()

,auction_id,experiment,hour,device_make,platform_os,browser,yes,no,day_of_week
0,0008ef63-77a7-448b-bd1e-075f42c55e39,1.0,8,46.0,6,2.0,0,0,0.0
1,000eabc5-17ce-4137-8efe-44734d914446,1.0,10,46.0,6,2.0,0,0,5.0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,1.0,2,29.0,6,3.0,0,1,3.0
3,00187412-2932-4542-a8ef-3633901c98d9,0.0,15,137.0,6,6.0,0,0,0.0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,0.0,15,46.0,6,2.0,0,0,0.0


In [40]:
# User who have answered the questioners 
ml_data =ml_data[np.logical_or(df['yes']==1,ml_data['no']==1)]
ml_data.shape

(1243, 9)

In [41]:
ml_data.head()

,auction_id,experiment,hour,device_make,platform_os,browser,yes,no,day_of_week
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,1.0,2,29.0,6,3.0,0,1,3.0
16,008aafdf-deef-4482-8fec-d98e3da054da,1.0,16,46.0,6,2.0,1,0,2.0
20,00a1384a-5118-4d1b-925b-6cdada50318d,1.0,8,46.0,6,2.0,0,1,1.0
23,00b6fadb-10bd-49e3-a778-290da82f7a8d,0.0,4,114.0,6,6.0,1,0,6.0
27,00ebf4a8-060f-4b99-93ac-c62724399483,0.0,15,46.0,6,2.0,0,1,0.0


In [42]:
ml_data = ml_data.reset_index(drop=True)

In [43]:
ml_data.shape

(1243, 9)

In [44]:
# Droping the user id value
del ml_data['auction_id']

In [45]:
ml_data.head()

,experiment,hour,device_make,platform_os,browser,yes,no,day_of_week
0,1.0,2,29.0,6,3.0,0,1,3.0
1,1.0,16,46.0,6,2.0,1,0,2.0
2,1.0,8,46.0,6,2.0,0,1,1.0
3,0.0,4,114.0,6,6.0,1,0,6.0
4,0.0,15,46.0,6,2.0,0,1,0.0


In [46]:
# 
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler()
ml_data[:] = scaling.fit_transform(ml_data[:])
ml_data.head()

,experiment,hour,device_make,platform_os,browser,yes,no,day_of_week
0,1.0,0.086957,0.098485,1.0,0.153846,0.0,1.0,0.500000
1,1.0,0.695652,0.162879,1.0,0.076923,1.0,0.0,0.333333
2,1.0,0.347826,0.162879,1.0,0.076923,0.0,1.0,0.166667
3,0.0,0.173913,0.420455,1.0,0.384615,1.0,0.0,1.000000
4,0.0,0.652174,0.162879,1.0,0.076923,0.0,1.0,0.000000


In [47]:
# Suffle data and split it to training set, validation set and testing set
import sklearn.utils
ml_data = sklearn.utils.shuffle(ml_data)
train, validate, test = np.split(ml_data.sample(frac=1), [int(.7*len(ml_data)), int(.9*len(ml_data))])
print('training set:', train.shape)
print('validation set:',validate.shape)
print('test set:', test.shape)

training set: (870, 8)
validation set: (248, 8)
test set: (125, 8)


In [48]:
X_train = train[['experiment', 'day_of_week', 'hour', 'device_make', 'platform_os', 'browser']]
Y_train = train['yes']

X_test= test[['experiment', 'day_of_week', 'hour', 'device_make', 'platform_os', 'browser']]
Y_test = test['yes'] 

X_val = validate[['experiment', 'day_of_week', 'hour', 'device_make', 'platform_os', 'browser']]
Y_val = validate['yes']